In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Flatten, Dense, Concatenate, Multiply,
    Dropout, BatchNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

C:\Users\soman\AppData\Roaming\Python\Python311\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\soman\AppData\Roaming\Python\Python311\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\soman\AppData\Roaming\Python\Python311\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please 

In [5]:
# --- Load Your Data ---
final_df = pd.read_csv('model_ready_data.csv')

# --- !! IMPORTANT: Feature Engineering Simulation !! ---
# Replace this block with your real feature calculations.

# 1. Simulate User Activity (e.g., number of interactions per user)
user_activity = final_df['user_idx'].value_counts().to_dict()
final_df['user_activity'] = final_df['user_idx'].map(user_activity)

# 2. Simulate Book Popularity (e.g., number of interactions per book)
book_popularity = final_df['book_idx'].value_counts().to_dict()
final_df['book_popularity'] = final_df['book_idx'].map(book_popularity)

# 3. Simulate Book Average Rating (e.g., a random float between 2.5 and 5)
# In your real code, you would calculate this from actual ratings data.
np.random.seed(42)
book_avg_ratings = {book_idx: np.random.uniform(2.5, 5.0) for book_idx in final_df['book_idx'].unique()}
final_df['book_avg_rating'] = final_df['book_idx'].map(book_avg_ratings)

# --- End of Simulation Block ---

# Define model constants
n_users = 11977
n_books = 36539
n_authors = 12080
embedding_size = 32
mlp_layers = [64, 32, 16]
n_numeric_features = 3 # We added 3 new features

In [6]:
# --- Input Layers ---
user_input = Input(shape=(1,), name='user_input')
book_input = Input(shape=(1,), name='book_input')
author_input = Input(shape=(1,), name='author_input')
# New input for our engineered numerical features
numeric_features_input = Input(shape=(n_numeric_features,), name='numeric_features_input')

# --- Embedding Layers (for GMF & MLP) ---
# GMF Embeddings
gmf_user_embedding = Embedding(input_dim=n_users, output_dim=embedding_size, name='gmf_user_embedding')(user_input)
gmf_book_embedding = Embedding(input_dim=n_books, output_dim=embedding_size, name='gmf_book_embedding')(book_input)
gmf_user_vector = Flatten()(gmf_user_embedding)
gmf_book_vector = Flatten()(gmf_book_embedding)

# MLP Embeddings
mlp_user_embedding = Embedding(input_dim=n_users, output_dim=embedding_size, name='mlp_user_embedding')(user_input)
mlp_book_embedding = Embedding(input_dim=n_books, output_dim=embedding_size, name='mlp_book_embedding')(book_input)
mlp_author_embedding = Embedding(input_dim=n_authors, output_dim=embedding_size, name='mlp_author_embedding')(author_input)
mlp_user_vector = Flatten()(mlp_user_embedding)
mlp_book_vector = Flatten()(mlp_book_embedding)
mlp_author_vector = Flatten()(mlp_author_embedding)

# --- GMF (Generalized Matrix Factorization) Path ---
gmf_output = Multiply()([gmf_user_vector, gmf_book_vector])

# --- Enhanced MLP (Multi-Layer Perceptron) Path ---
# Concatenate all MLP inputs: embeddings + numerical features
mlp_concat = Concatenate()([
    mlp_user_vector,
    mlp_book_vector,
    mlp_author_vector,
    numeric_features_input  # <-- NEW
])

# Build MLP layers dynamically with Dropout and BatchNormalization
mlp = mlp_concat
for layer_size in mlp_layers:
    mlp = Dense(layer_size, activation='relu')(mlp)
    mlp = BatchNormalization()(mlp)  # <-- NEW: Stabilizes training
    mlp = Dropout(0.3)(mlp)          # <-- NEW: Prevents overfitting
mlp_output = mlp

# --- Fusion (NeuMF) Layer ---
neumf_concat = Concatenate()([gmf_output, mlp_output])
output = Dense(1, activation='sigmoid', name='output')(neumf_concat)

# --- Create the Model ---
model = Model(
    inputs=[user_input, book_input, author_input, numeric_features_input], # <-- UPDATED
    outputs=output,
    name='Enhanced_Hybrid_NCF_Model'
)

# --- Compile the Model ---
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()

Model: "Enhanced_Hybrid_NCF_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ book_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ author_input        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_user_embedding  │ (None, 1, 32)     │    383,264 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_book_embedding  │ (None, 1, 32)     │  1,169,248 │ book_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_author_embeddi… │ (None, 1, 32)     │    386,560 │ author_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 32)        │          0 │ mlp_user_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 32)        │          0 │ mlp_book_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 32)        │          0 │ mlp_author_embed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric_features_i… │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 99)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0],  │
│                     │                   │            │ flatten_4[0][0],  │
│                     │                   │            │ numeric_features… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      6,400 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64)        │        256 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32)        │        128 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gmf_user_embedding  │ (None, 1, 32)     │    383,264 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 3,501,089 (13.36 MB)

 Trainable params: 3,500,865 (13.35 MB)

 Non-trainable params: 224 (896.00 B)

In [7]:
# --- 1. Split the DataFrame ---
train_df, val_df = train_test_split(final_df, test_size=0.2, random_state=42)

# --- 2. Scale Numerical Features ---
# It's crucial to scale features for a neural network
numeric_cols = ['user_activity', 'book_popularity', 'book_avg_rating']
scaler = StandardScaler()

# Fit on training data and transform both sets
X_train_numeric = scaler.fit_transform(train_df[numeric_cols])
X_val_numeric = scaler.transform(val_df[numeric_cols])

# --- 3. Create Training and Validation Sets ---
# X_train is now a list of 4 arrays
X_train = [
    train_df['user_idx'].values,
    train_df['book_idx'].values,
    train_df['author_idx'].values,
    X_train_numeric  # <-- NEW
]
y_train = train_df['interaction'].values

# X_val is also a list of 4 arrays
X_val = [
    val_df['user_idx'].values,
    val_df['book_idx'].values,
    val_df['author_idx'].values,
    X_val_numeric  # <-- NEW
]
y_val = val_df['interaction'].values

# --- 4. Handle Class Imbalance ---
# This will give more importance to the rare '1' class
weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), weights))

print(f"Calculated Class Weights: {class_weight_dict}")

Calculated Class Weights: {0: 0.6247979332537656, 1: 2.503238302766176}


In [8]:
# --- Define Callbacks ---

# 1. Stop training if 'val_loss' doesn't improve for 3 consecutive epochs
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True  # Restores model weights from the epoch with the best 'val_loss'
)

# 2. Reduce learning rate if 'val_loss' plateaus for 2 epochs
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Reduce LR by half
    patience=2,
    min_lr=0.00001, # Don't let the LR get too small
    verbose=1
)

# 3. Save only the best performing model
model_checkpoint = ModelCheckpoint(
    "best_enhanced_ncf_model.h5",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# --- 4. Train the Model ---
print("\nStarting enhanced model training...")
history = model.fit(
    x=X_train,  # Pass the list of 4 input arrays
    y=y_train,
    batch_size=256,
    epochs=30,  # Set a high number, EarlyStopping will find the optimal one
    verbose=1,
    validation_data=(X_val, y_val),
    class_weight=class_weight_dict,  # <-- NEW: Apply class weights
    callbacks=[early_stopping, reduce_lr, model_checkpoint] # <-- NEW: Add callbacks
)

print("✅ Model training complete.")


Starting enhanced model training...
Epoch 1/30
2908/2908 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.7716 - auc: 0.8012 - loss: 0.5331
Epoch 1: val_loss improved from None to 0.45319, saving model to best_enhanced_ncf_model.h5


2908/2908 ━━━━━━━━━━━━━━━━━━━━ 127s 43ms/step - accuracy: 0.7932 - auc: 0.8287 - loss: 0.5041 - val_accuracy: 0.7987 - val_auc: 0.8540 - val_loss: 0.4532 - learning_rate: 0.0010
Epoch 2/30
2907/2908 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8150 - auc: 0.8839 - loss: 0.4311
Epoch 2: val_loss improved from 0.45319 to 0.44011, saving model to best_enhanced_ncf_model.h5


2908/2908 ━━━━━━━━━━━━━━━━━━━━ 128s 44ms/step - accuracy: 0.8266 - auc: 0.8904 - loss: 0.4201 - val_accuracy: 0.8091 - val_auc: 0.8499 - val_loss: 0.4401 - learning_rate: 0.0010
Epoch 3/30
2908/2908 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.9359 - auc: 0.9794 - loss: 0.2216
Epoch 3: val_loss did not improve from 0.44011
2908/2908 ━━━━━━━━━━━━━━━━━━━━ 144s 50ms/step - accuracy: 0.9439 - auc: 0.9832 - loss: 0.2013 - val_accuracy: 0.8077 - val_auc: 0.8182 - val_loss: 0.4667 - learning_rate: 0.0010
Epoch 4/30
2908/2908 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9875 - auc: 0.9987 - loss: 0.0675
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 4: val_loss did not improve from 0.44011
2908/2908 ━━━━━━━━━━━━━━━━━━━━ 133s 46ms/step - accuracy: 0.9865 - auc: 0.9985 - loss: 0.0649 - val_accuracy: 0.8057 - val_auc: 0.7977 - val_loss: 0.5398 - learning_rate: 0.0010
Epoch 5/30
2907/2908 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9969 - auc: 0.9998 -

In [9]:
print("\nEvaluating model performance on validation set...")

# Predict probabilities
y_pred_prob = model.predict(X_val)

# Convert to binary (0 or 1) using 0.5 as the threshold
y_pred = (y_pred_prob > 0.5).astype(int)

# --- Compute Metrics ---
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, zero_division=0)
recall = recall_score(y_val, y_pred, zero_division=0)
f1 = f1_score(y_val, y_pred, zero_division=0)
cm = confusion_matrix(y_val, y_pred)

# --- Print All Metrics ---
print("\n--- Performance Metrics ---")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

print("\nClassification Report:\n", classification_report(y_val, y_pred, zero_division=0))

print("\nConfusion Matrix:")
print(cm)


Evaluating model performance on validation set...
5816/5816 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step

--- Performance Metrics ---
Accuracy:  0.8091
Precision: 0.5181
Recall:    0.7210
F1 Score:  0.6029

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.83      0.87    148689
           1       0.52      0.72      0.60     37413

    accuracy                           0.81    186102
   macro avg       0.72      0.78      0.74    186102
weighted avg       0.84      0.81      0.82    186102


Confusion Matrix:
[[123603  25086]
 [ 10440  26973]]


In [10]:
import joblib

# --- 1. Load the Best Model (to confirm it works) ---
try:
    model = tf.keras.models.load_model('best_enhanced_ncf_model.h5')
    print("✅ Successfully loaded 'best_enhanced_ncf_model.h5'")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure model training completed successfully and the file exists.")

# --- 2. Save the Scaler ---
# 'scaler' is the StandardScaler object you fit on the training data
joblib.dump(scaler, 'standard_scaler.joblib')
print("✅ Saved 'standard_scaler.joblib'")

# --- 3. Create and Save Book Metadata Lookup ---
# We need a file that maps a 'book_idx' to its author and features.
# We'll create this from the 'final_df', dropping duplicates.
book_metadata_cols = [
    'book_idx',
    'author_idx',
    'book_popularity',
    'book_avg_rating'
]

# Get unique book information
book_metadata_df = final_df[book_metadata_cols].drop_duplicates(subset=['book_idx']).set_index('book_idx')
book_metadata_df.to_csv('book_metadata.csv')
print("✅ Saved 'book_metadata.csv'")

# --- 4. Create and Save User Feature Lookup ---
# We also need the 'user_activity' feature for a given user
user_metadata_cols = ['user_idx', 'user_activity']
user_metadata_df = final_df[user_metadata_cols].drop_duplicates(subset=['user_idx']).set_index('user_idx')
user_metadata_df.to_csv('user_metadata.csv')
print("✅ Saved 'user_metadata.csv'")

print("\nAll necessary artifacts are saved! 🚀")

✅ Successfully loaded 'best_enhanced_ncf_model.h5'
✅ Saved 'standard_scaler.joblib'
✅ Saved 'book_metadata.csv'
✅ Saved 'user_metadata.csv'

All necessary artifacts are saved! 🚀


In [11]:
# --- Cell 8: Create the Final 'book_title_lookup.csv' Bridge File ---

print("Creating book title lookup file...")

# 1. Load your original books file (BX-Books.csv)
try:
    books_csv_path = 'BX-Books.csv'
    original_books_df = pd.read_csv(
        books_csv_path, 
        encoding='latin1', 
        sep=';', 
        on_bad_lines='skip'
    )
except FileNotFoundError:
    print(f"Error: Could not find '{books_csv_path}'. Make sure it's in the same directory.")
    # Handle error appropriately

# 2. Select the *original* title/author columns + the 'ISBN' to link them
original_books_df = original_books_df[['ISBN', 'Book-Title', 'Book-Author']]
original_books_df = original_books_df.drop_duplicates(subset=['ISBN'])

# 3. Get the 'book_idx' and 'ISBN' from your final_df
#    This ASSUMES 'ISBN' is in your 'model_ready_data.csv' file!
if 'ISBN' in final_df.columns:
    idx_to_isbn_df = final_df[['book_idx', 'ISBN']].drop_duplicates()
    
    # 4. Merge the two DataFrames to create the bridge:
    #    [book_idx] <-> [ISBN] <-> [Book-Title]
    book_title_lookup_df = pd.merge(
        idx_to_isbn_df,
        original_books_df,
        on='ISBN'
    )
    
    # 5. Save the final lookup file
    #    This file will have [book_idx, ISBN, Book-Title, Book-Author]
    book_title_lookup_df.to_csv('book_title_lookup.csv', index=False, encoding='latin1')
    print("✅ Successfully created 'book_title_lookup.csv'.")
    print("You are now ready to run the recommendation script.")
    
else:
    print("\n--- CRITICAL ERROR ---")
    print("Your 'model_ready_data.csv' file does not contain the 'ISBN' column.")
    print("I cannot create the bridge file.")
    print("Please go back to your *original preprocessing* notebook and create a file that maps 'book_idx' to 'Book-Title' and 'Book-Author'.")

Creating book title lookup file...

--- CRITICAL ERROR ---
Your 'model_ready_data.csv' file does not contain the 'ISBN' column.
I cannot create the bridge file.
Please go back to your *original preprocessing* notebook and create a file that maps 'book_idx' to 'Book-Title' and 'Book-Author'.


C:\Users\soman\AppData\Local\Temp\ipykernel_10816\1438804685.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  original_books_df = pd.read_csv(
